### Step 0: Preprocess Data

In [13]:
# Step 0 Data Processing

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels

In [15]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
X_train.shape

(55000, 784)

In [17]:
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [18]:
y_train.shape

(55000, 10)

In [0]:
X_train = X_train.reshape(-1,28,28)

In [0]:
X_test = X_test.reshape(-1,28,28)

In [23]:
X_train.shape

(55000, 28, 28)

- Each image is 28 pixels (rows) by 28 pixels (cols). 
- We treat each image as a sequence of data, that is, 
    - the first row is the first step; 
    - second row is the second step and so on. 
- Therefore, n_steps = number of rows and n_inputs = number of columns.

In [0]:
# Step 1: Initial Setup

# parameters
n_steps = 28 # 28 rows
n_inputs = 28 # 28 cols
n_outputs = 10 # 10 classes

import tensorflow as tf
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32)

In [0]:
# Step 2 Model

cell = tf.nn.rnn_cell.LSTMCell(64, name='1')
outputs, final_output_state = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

In [49]:
outputs.shape

TensorShape([Dimension(None), Dimension(28), Dimension(64)])

In [0]:
Ylogits = tf.layers.dense(outputs[:,-1], n_outputs)
yhat = tf.nn.softmax(Ylogits)

In [29]:
Ylogits

<tf.Tensor 'dense_5/BiasAdd:0' shape=(?, 10) dtype=float32>

In [0]:
# Step 3 Loss Function

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=Ylogits))

In [0]:
# Step 4 Optimizer

lr = 0.01
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [32]:
is_correct = tf.equal(tf.argmax(y,1),tf.argmax(yhat,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

training_epochs = 5
batch_size = 100
# Step 5: Training Loop
for epoch in range(training_epochs):
    for i in range(int(X_train.shape[0] / batch_size)):
        batch_X = X_train[(i*batch_size):((i+1)*batch_size)]
        batch_y = y_train[(i*batch_size):((i+1)*batch_size)]
        train_data = {X: batch_X, y: batch_y}
        sess.run(train, feed_dict=train_data)
        if i%100 == 0:
            print(epoch, "/", i,"Training Accuracy = ", sess.run(accuracy, feed_dict=train_data))

0 / 0 Training Accuracy =  0.31
0 / 100 Training Accuracy =  0.86
0 / 200 Training Accuracy =  0.91
0 / 300 Training Accuracy =  0.97
0 / 400 Training Accuracy =  0.98
0 / 500 Training Accuracy =  1.0
1 / 0 Training Accuracy =  0.94
1 / 100 Training Accuracy =  0.98
1 / 200 Training Accuracy =  0.97
1 / 300 Training Accuracy =  1.0
1 / 400 Training Accuracy =  0.97
1 / 500 Training Accuracy =  0.98
2 / 0 Training Accuracy =  0.96
2 / 100 Training Accuracy =  0.99
2 / 200 Training Accuracy =  0.99
2 / 300 Training Accuracy =  0.97
2 / 400 Training Accuracy =  0.96
2 / 500 Training Accuracy =  0.98
3 / 0 Training Accuracy =  0.96
3 / 100 Training Accuracy =  0.99
3 / 200 Training Accuracy =  0.98
3 / 300 Training Accuracy =  0.98
3 / 400 Training Accuracy =  0.99
3 / 500 Training Accuracy =  0.99
4 / 0 Training Accuracy =  0.96
4 / 100 Training Accuracy =  1.0
4 / 200 Training Accuracy =  0.98
4 / 300 Training Accuracy =  0.98
4 / 400 Training Accuracy =  1.0
4 / 500 Training Accuracy = 

In [0]:
test_predict = sess.run(yhat, feed_dict = {X: X_test})

In [34]:
test_predict

array([[5.6176368e-06, 3.0691732e-05, 4.7508362e-05, ..., 9.9955207e-01,
        8.6718347e-07, 2.1413379e-04],
       [3.3822907e-06, 3.4865843e-05, 9.9989974e-01, ..., 7.0431302e-06,
        8.5273832e-06, 4.4475609e-08],
       [4.6159311e-08, 9.9998426e-01, 2.3385553e-06, ..., 5.3373365e-06,
        3.0774459e-06, 9.6910171e-08],
       ...,
       [1.4778474e-04, 3.5134890e-06, 1.8354391e-05, ..., 9.8387565e-05,
        8.6480053e-05, 3.1842829e-03],
       [2.3074585e-05, 2.0222382e-05, 5.2204145e-05, ..., 4.5608599e-06,
        1.4532649e-02, 3.1379619e-04],
       [1.7901724e-04, 1.0497151e-05, 2.5168719e-04, ..., 4.9674245e-08,
        1.9207649e-04, 7.6062440e-07]], dtype=float32)

In [35]:
sess.run(tf.argmax(test_predict,1))

array([7, 2, 1, ..., 4, 5, 6])

In [36]:
sess.run(accuracy, feed_dict = {X: X_test, y: y_test})

0.9807

In [0]:
acc = []
for i in range(int(X_test.shape[0] / batch_size)):
    batch_X = X_test[(i*batch_size):((i+1)*batch_size)]
    batch_y = y_test[(i*batch_size):((i+1)*batch_size)]
    test_data = {X: batch_X, y: batch_y}
    sess.run(train, feed_dict = test_data)
    acc.append(sess.run(accuracy, feed_dict = test_data))

In [38]:
acc

[0.99,
 1.0,
 0.99,
 1.0,
 0.98,
 0.99,
 0.98,
 0.98,
 1.0,
 0.99,
 0.99,
 0.99,
 0.97,
 0.99,
 1.0,
 1.0,
 0.99,
 0.98,
 1.0,
 0.98,
 1.0,
 0.97,
 0.98,
 0.98,
 0.95,
 0.98,
 0.99,
 0.99,
 0.98,
 0.98,
 0.95,
 1.0,
 0.98,
 0.99,
 0.97,
 0.98,
 0.99,
 0.98,
 0.96,
 1.0,
 0.99,
 0.99,
 0.98,
 1.0,
 0.98,
 0.98,
 0.99,
 0.98,
 0.97,
 1.0,
 1.0,
 0.98,
 0.99,
 1.0,
 0.99,
 1.0,
 1.0,
 1.0,
 0.99,
 0.96,
 0.98,
 0.99,
 1.0,
 1.0,
 0.99,
 0.94,
 0.99,
 0.99,
 1.0,
 0.99,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.99,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.98,
 0.99,
 0.99,
 0.98,
 1.0,
 1.0,
 1.0,
 1.0,
 0.96,
 1.0,
 1.0,
 1.0,
 0.98,
 1.0,
 0.98,
 0.97,
 0.98,
 0.99]